# Stress Constrained TO Algorithm

Conceptualised and developed by erin.yu22@imperial.ac.uk, sasha.halsey20@imperial.ac.uk and a.panesar@imperial.ac.uk

-------------------
### Objectives of this lab

1) Add a stress constraint to the TO problem
2) Compare to previous TO results

####  Import library

Run the cell below to import necessarry python libraries for this lab.

If you encounter an error, identify the unsuccessfully-installed library name by reading the error message, cut the corresponding pip install statement (without '#' symbol) into a separate cell and run it. Upon running the pip install cell, then restart the kernel and run the (default) import library cell again. Raise hand during the lab or email erin.yu22@imperial.ac.uk if you cannot resolve the issue.

In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.sparse import coo_matrix, lil_matrix
from scipy.sparse.linalg import spsolve

-----------
# 1) Adding Another Constraint to the Problem
### Optimisation Framework

Previously we solved a density–based topology optimisation problem with the SIMP formulation.

- **Design variable:**  
  Elemental relative density  
  $$ x_e \in [0,1] $$

- **Objective function:**  
  Minimise compliance (maximise stiffness)
  $$
  \min_x\; C(x) = \mathbf{F}^\mathsf{T}\mathbf{U}(x)
  $$
  where the displacement field satisfies  
  $$
  \mathbf{K}(x)\,\mathbf{U}(x) = \mathbf{F}.
  $$

- **Constraint:**  
  Global volume fraction  
  $$
  \frac{1}{N_e}\sum_{e=1}^{N_e} x_e = V^\ast.
  $$

Now we add a second constraint based on the mechanical stress of the elements. It would be costly however to check every element and limit it's stress to a maximum value, and this search isn't differentiable.

Instead we use an aggregation technique that picks up and smooths out the maximum stressed elements, giving us a value that we can reduce by adding a limiting factor and incoorperating it as a constraint into our problem. This function integrates (aggregates) all the stresses and penalises the higher values with a factor called 'p-norm' in a similar way to SIMP:

$$
p_{\text{stress}} = \left( \frac{1}{N_\text{el}} \sum_{e=1}^{N_\text{el}} \sigma_{\text{vm},e}^{\,p_\text{norm}} \right)^{\frac{1}{p_\text{norm}}}
$$

where:  

- $N_\text{el}$ = total number of finite elements  
- $\sigma_{\text{vm},e}$ = von Mises stress of element $e$  
- $p_\text{norm}$ = chosen p-norm exponent (typically 8–20), but we will play around with this later.

The p-norm stress constraint is formulated as an inequality:

$$
c_2 = p_{\text{stress}} - \sigma_\text{allow} \le 0
$$

where $\sigma_\text{allow}$ is the allowed stress (e.g. 90% of maximum).  
This ensures that the structural stress does not exceed the specified limit.


# 1.1) Input parameters

In [ ]:
nelx = 30
nely = 20
nel = nelx * nely
volfrac = 0.3
penal = 3.0
rmin = 1.0

Emin = 1e-9
Emax = 1.0
nu = 0.3

# stress p-norm parameters
p_norm = 8.0
stress_weight = 1e-3      # small weight prevents collapse; tune upward if needed
eps_vm, eps_S = 1e-12, 1e-12          # to avoid zero division

# plotting functions
def plot_density(xPhys):
    fig, ax = plt.subplots(figsize=(5,3))
    im = ax.imshow(-xPhys.reshape((nelx, nely)).T, cmap='gray', interpolation='none',
                   norm=colors.Normalize(vmin=-1, vmax=0), origin='lower')
    ax.set_xticks([]); ax.set_yticks([])
    plt.title("Density")
    plt.show()

def plot_vm(sigma_vm):
    sigma_vm_grid = sigma_vm.reshape((nelx, nely)).T
    fig, ax = plt.subplots(figsize=(5,3))
    im = ax.imshow(sigma_vm_grid, cmap="inferno", interpolation='none', origin='lower')
    ax.set_xticks([]); ax.set_yticks([])
    plt.title("Von Mises stress (element-wise)")
    plt.colorbar(im, ax=ax, fraction=0.03, pad=0.02)
    plt.tight_layout()
    plt.show()

# 1.2) FE Function Defined
(This is the same as the previous function showed in the lab, but cleaned up and call-able)

In [ ]:
from functions import lk, FE_solve, compliance_and_sensitivity, oc


KE = lk(E=1.0, nu=nu)
ndof = 2 * (nelx + 1) * (nely + 1)

edofMat = np.zeros((nel, 8), dtype=int)
for elx in range(nelx):
    for ely in range(nely):
        el = ely + elx*nely
        n1 = (nely+1)*elx + ely
        n2 = (nely+1)*(elx+1) + ely
        edofMat[el, :] = np.array([
            2*n1+2, 2*n1+3, 2*n2+2, 2*n2+3,
            2*n2,   2*n2+1, 2*n1,   2*n1+1
        ])
dofs = np.arange(ndof)
fixed = np.union1d(dofs[0:2*(nely+1):2],
                   np.array([2*(nelx+1)*(nely+1)-1]))
free = np.setdiff1d(dofs, fixed)

# Boundary conditions
f = np.zeros(ndof)
f[1] = -1.0

nfilter = int(nelx*nely*((2*(np.ceil(rmin)-1)+1)**2))
iH = np.zeros(nfilter)
jH = np.zeros(nfilter)
sH = np.zeros(nfilter)
cc = 0

for i in range(nelx):
    for j in range(nely):
        row = i*nely + j
        kk1 = int(max(i - (np.ceil(rmin)-1), 0))
        kk2 = int(min(i + np.ceil(rmin), nelx))
        ll1 = int(max(j - (np.ceil(rmin)-1), 0))
        ll2 = int(min(j + np.ceil(rmin), nely))
        for k in range(kk1, kk2):
            for l in range(ll1, ll2):
                col = k*nely + l
                fac = rmin - np.sqrt((i-k)**2 + (j-l)**2)
                iH[cc] = row
                jH[cc] = col
                sH[cc] = max(0.0, fac)
                cc += 1

H = coo_matrix((sH, (iH, jH)), shape=(nel, nel)).tocsc()
Hs = H.sum(1)

x = volfrac * np.ones(nel, dtype=float)
xold = x.copy()
xPhys = x.copy()
dv = np.ones(nel)
ce = np.ones(nel)

g=0 # must be initialised to use the NGuyen/Paulino OC approach
dc=np.zeros((nely,nelx), dtype=float)

# Set loop counter and gradient vectors 
loop=0
change=1

# 1.6) Stress Calculation and p-norm Sensitivity

We use $c_2$ to define our second constraint, p-norm. You can also change sigma_max to relax or increase the 'allowed' p-norm aggregation. However, make sure to run it as-is first to see the magnitude of the p-norm aggregation, to ensure you are over-constraining it.

In [ ]:
D_mat_unit = (1.0 / (1.0 - nu**2)) * np.array([[1.0, nu, 0.0],
                                               [nu, 1.0, 0.0],
                                               [0.0, 0.0, (1.0-nu)/2.0]])
B_center = (1.0/4.0) * np.array([
    [-1,  0,  1,  0,  1,  0, -1,  0],
    [ 0, -1,  0, -1,  0,  1,  0,  1],
    [-1, -1, -1,  1,  1,  1,  1, -1]])

sigma_max = 0.3                   # allowable p-norm stress limit - change this value as needed

def stress_and_pnorm_sensitivity(U, xPhys, penal, p_norm, sigma_max, eps_vm=1e-12, eps_S=1e-12):
    sigma_elem = np.zeros((nel, 3))
    sigma_vm = np.zeros(nel)
    eps_elem = np.zeros((nel, 3))
    for e in range(nel):
        u_e = U[edofMat[e, :]]
        eps_e = B_center @ u_e
        eps_elem[e, :] = eps_e
        E_e = Emin + xPhys[e]**penal * (Emax - Emin)
        sig_e = (E_e * D_mat_unit) @ eps_e
        sigma_elem[e, :] = sig_e
        sx, sy, txy = sig_e
        sigma_vm[e] = np.sqrt(sx**2 - sx*sy + sy**2 + 3.0*(txy**2))

    # safe p-norm
    vm_safe = np.maximum(sigma_vm, eps_vm)
    S = np.sum(vm_safe**p_norm) / nel + eps_S
    p_stress = S**(1.0/p_norm)

    # constraint c2
    c2 = p_stress - sigma_max         # constraint: must satisfy c2 <= 0

    # sensitivity dc2
    dc2 = np.zeros(nel)
    dE_dx = penal * xPhys**(penal-1) * (Emax - Emin)
    prefactor = (1.0/nel) * S**(1.0/p_norm - 1.0)

    for e in range(nel):
        dsig_dE = D_mat_unit @ eps_elem[e]
        dsig_dx = dsig_dE * dE_dx[e]
        sx, sy, txy = sigma_elem[e]
        vm = vm_safe[e]
        # avoid division by zero for vm (we used vm_safe)
        dvm_dsig = np.array([
            (2.0*sx - sy) / (2.0*vm),
            (2.0*sy - sx) / (2.0*vm),
            (3.0*txy) / vm])
        dvm_dx = dvm_dsig @ dsig_dx
        dc2[e] = prefactor * (vm**(p_norm-1.0)) * dvm_dx

    # filter
    dc2 = np.asarray(H * (dc2[np.newaxis].T / Hs))[:, 0]
    return sigma_vm, p_stress, c2, dc2

# 1.7) Optimality Criterion Update

In [ ]:
def oc(nelx, nely, x, volfrac, dc, dv, g):
    l1 = 0.0; l2 = 1e9; move = 0.2
    xnew = np.zeros(nelx*nely)
    for _ in range(80):
        lmid = 0.5*(l2 + l1)
        denom = -dc/dv / lmid
        with np.errstate(divide='ignore', invalid='ignore'):
            arg = np.maximum(1e-9, denom)
            x_candidate = np.maximum(1e-3, np.maximum(x - move, np.minimum(1.0, np.minimum(x + move, x * np.sqrt(arg)))))
        gt = g + np.sum(dv*(x_candidate - x))
        if gt > 0:
            l1 = lmid
        else:
            l2 = lmid
        xnew[:] = x_candidate[:]
        if abs(l2 - l1) / (l1 + l2 + 1e-9) < 1e-3:
            break
    return xnew, gt

# 1.8) Iterate a Number of Times

In [ ]:
max_iter = 50
g = 0.0

for it in range(max_iter):
    # FE
    U = FE_solve(nelx, nely, xPhys, penal, KE, free, f)

    # compliance and dc
    obj, dc, ce = compliance_and_sensitivity(U, KE, penal, xPhys)

    # stress and dc2
    sigma_vm, pnorm_val, c2, dc2 = stress_and_pnorm_sensitivity(U, xPhys, penal, p_norm, sigma_max, eps_vm, eps_S)

    # combine sensitivities (use stress_weight small to avoid void collapse)
    # dc_total = dc + stress_weight * dc2
    dc_total = dc + stress_weight * (dc2 * np.sign(c2))


    # volume sensitivity (filtered)
    dv = np.asarray(H * (np.ones(nel)[np.newaxis].T / Hs))[:, 0]

    # OC update
    xold[:] = x
    x[:], g = oc(nelx, nely, x, volfrac, dc_total, dv, g)
    xPhys[:] = np.asarray(H * x[np.newaxis].T / Hs)[:, 0]

    change = np.max(np.abs(x - xold))
    print(f"it {it:3d}   obj: {obj:.6f}   pnorm: {pnorm_val:.6f}   change: {change:.6f}")
    print(f"          stress_constraint c2 = {c2:.6f}  (OK? {c2 <= 0})")

    if change < 1e-3:
        break

# 1.9) Plot

In [ ]:
print(f"Average density: {np.mean(xPhys):.4f}")
plot_density(xPhys)
plot_vm(sigma_vm)

----------
# 2) Comparing - Your Turn!

Now try running again, but changing parameters such as p_norm or sigma_max to see how the convergence and design changes. 
Keep an eye on the values of p_norm aggregation, c2, obj, as you may need more iterations or relaxing of constraints to ensure convergence.

![ParetoFront](attachment:ParetoFront.png)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---- Fake data generation ----
# Normalised p-norm stress constraint (0 = relaxed, 1 = very tight)
x = np.linspace(0, 1, 20)

# Fake Pareto-compliant decreasing trend + small noise
y = 1.0 - 0.7*x + 0.05*np.random.rand(len(x))

# Sort points to create a clean monotonic Pareto front
# (lower compliance is better → keep decreasing order)
idx = np.argsort(x)
x_sorted = x[idx]
y_sorted = y[idx]

# Enforce monotonic downward trend (fake but visually correct)
y_pareto = np.minimum.accumulate(y_sorted)

# ---- Plotting ----
fig, ax = plt.subplots(figsize=(5,4))

# scatter all points
ax.scatter(x_sorted, y_sorted, color='lightgray', label='Sampled designs')

# Pareto front
ax.plot(x_sorted, y_pareto, linewidth=2, label='Pareto front')

# labels and title
ax.set_xlabel('Normalised p-norm Stress Constraint')
ax.set_ylabel('Normalised Compliance Objective')
ax.set_title('Pareto Front')

ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
max_iter = 50
g = 0.0

p_norm = 10.0
sigma_max = 0.4

for it in range(max_iter):
    # FE
    U = FE_solve(nelx, nely, xPhys, penal, KE, free, f)

    # compliance and dc
    obj, dc, ce = compliance_and_sensitivity(U, KE, penal, xPhys)

    # stress and dc2
    sigma_vm, pnorm_val, c2, dc2 = stress_and_pnorm_sensitivity(U, xPhys, penal, p_norm, sigma_max, eps_vm, eps_S)

    # combine sensitivities (use stress_weight small to avoid void collapse)
    # dc_total = dc + stress_weight * dc2
    dc_total = dc + stress_weight * (dc2 * np.sign(c2))


    # volume sensitivity (filtered)
    dv = np.asarray(H * (np.ones(nel)[np.newaxis].T / Hs))[:, 0]

    # OC update
    xold[:] = x
    x[:], g = oc(nelx, nely, x, volfrac, dc_total, dv, g)
    xPhys[:] = np.asarray(H * x[np.newaxis].T / Hs)[:, 0]

    change = np.max(np.abs(x - xold))
    print(f"it {it:3d}   obj: {obj:.6f}   pnorm: {pnorm_val:.6f}   change: {change:.6f}")
    print(f"          stress_constraint c2 = {c2:.6f}  (OK? {c2 <= 0})")

    if change < 1e-3:
        break

print(f"Average density: {np.mean(xPhys):.4f}")
plot_density(xPhys)
plot_vm(sigma_vm)